In [626]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [627]:
df_train=pd.read_csv('dataset/train.csv') 
df_train.Image=[img.split('.')[0] for img in df_train.Image] 
labels=pd.read_csv('df_train.csv') 
labels.columns=['Image']+['{}'.format(i) for i in range(1,labels.shape[1])]
df_train.Image=df_train.Image.astype('int')
labels.Image=labels.Image.astype('int')
df=pd.merge(labels,df_train,on=['Image']).iloc[:,1:]
df.head()

,1,2,3,4,5,6,7,8,9,10,...,16377,16378,16379,16380,16381,16382,16383,16384,16385,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.401423,2.109968,0.0,3.244040,0.0,9,kathak
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.804171,1.117670,0.0,1.540733,0.0,3,kathak
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.286839,0.648386,0.0,0.365375,0.0,3,manipuri
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.465765,0.000000,0.0,0.000000,0.0,8,manipuri
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.878799,1.082086,0.0,1.526092,0.0,3,manipuri


In [628]:
_=list(enumerate(list(df.target.value_counts().index))) 
rev_labels=dict(_) 
labels=dict([(v,k) for (k,v) in _])

In [629]:
for i in range(len(df)):
    df.iloc[i,-1]=labels[df.iloc[i,-1]] 
df.target=df.target.astype('int')

In [630]:
# One Hot Encoder 
def onehot(df,features):
    df=df.copy()
    for feature in features : 
        cats=list(np.sort(list(df.loc[:,feature].unique())))
        if len(cats)==1 :
            continue 
            
        loc=df.columns.get_loc(feature) 
        [df.insert(loc=loc,column='{}_{}'.format(feature,cat),value=0.0) for cat in cats[:-1]]  
        for index in df.index :
            val=df.loc[index,feature] 
            clm='{}_{}'.format(feature,val) 
            if clm in df.columns :
                df.loc[index,clm]=1.0 
            
        df=df.drop(columns=[feature]) 
        
    
        
    return df

In [631]:
df=onehot(df,['16385'])

In [632]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,16385_8,16385_7,16385_6,16385_5,16385_4,16385_3,16385_2,16385_1,16385_0,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7


**Train-Test Split**

In [633]:
df=df.sample(frac=1.0)
split=int(0.80*len(df))
df_train=df.iloc[:split,:]
df_test=df.iloc[split:,:]

**Model & Training**

In [657]:
model=SVC(C=5.0)
model.fit(df_train.iloc[:,:-1],df_train.iloc[:,-1:])

/home/gajesh/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=5.0)

In [658]:
model.score(df_train.iloc[:,:-1],df_train.iloc[:,-1:])

0.9725085910652921

In [659]:
model.score(df_test.iloc[:,:-1],df_test.iloc[:,-1:])

0.4520547945205479

In [644]:
np.argsort(model.feature_importances_)[::-1][:100]

AttributeError: 'SVC' object has no attribute 'feature_importances_'

In [186]:
df.columns[960]

'961'